In [1]:
# Vizuelna podešavanja okruženja (samo razvuče notebook na širinu ekrana)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [2]:
import networkx as nx
import pickle

with open('data_cleaned/data2018_cleaned', 'rb') as file:
    data2018 = pickle.load(file)
with open('data_cleaned/data2019_cleaned', 'rb') as file:
    data2019 = pickle.load(file)
with open('data_cleaned/data2020_cleaned', 'rb') as file:
    data2020 = pickle.load(file)

data = data2018.append(data2019.append(data2020))

with open("data_cleaned/all_data_cleaned", "wb") as file:
    pickle.dump(data, file)



In [3]:
lista_winners2018 = set(data2018['winner_id'].unique())
lista_losers2018 = set(data2018['loser_id'].unique())

lista_winners2019 = set(data2019['winner_id'].unique())
lista_losers2019 = set(data2019['loser_id'].unique())

lista_winners2020 = set(data2020['winner_id'].unique())
lista_losers2020 = set(data2020['loser_id'].unique())

players2018 = lista_winners2018.union(lista_losers2018)
players2019 = lista_winners2019.union(lista_losers2019)
players2020 = lista_winners2020.union(lista_losers2020)

svi_igraci = players2018.union(players2019.union(players2020))
print(f"Ukupan broj igraca je {len(svi_igraci)}")

Ukupan broj igraca je 581


In [4]:
import pandas as pd

dataPath = 'data/atp_players.csv'
players = pd.read_csv(dataPath)

dataPath = 'data/atp_rankings_current.csv'
rankings = pd.read_csv(dataPath)

In [5]:
G = nx.Graph()
for player_id in svi_igraci:
    player = players[players['player_id'] == player_id]
    if player.empty:
        print("Postoji id igraca koji ne postoji u listi svih igraca!")
    else:
        player_rank = rankings[(rankings['player_id'] == player_id)]
        rank_p = 0
        points_p = 0
        
        if not player_rank.empty:
            last_row = player_rank.shape[0] - 1
            rank_p = int(player_rank.iloc[last_row, 1])
            points_p = int(player_rank.iloc[last_row, 3])
            
        G.add_node(str(player_id), first_name = str(player.iloc[0, 1]), last_name = str(player.iloc[0, 2]), country = str(player.iloc[0, 5]), hand = str(player.iloc[0, 3]), rank = rank_p, points = points_p)


In [6]:
data_reduced2018 = data2018[['winner_id', 'loser_id']]
data_reduced2018 = data_reduced2018.astype({'winner_id': str, 'loser_id': str})

data_reduced2019 = data2019[['winner_id', 'loser_id']]
data_reduced2019 = data_reduced2019.astype({'winner_id': str, 'loser_id': str})

data_reduced2020 = data2020[['winner_id', 'loser_id']]
data_reduced2020 = data_reduced2020.astype({'winner_id': str, 'loser_id': str})

for _, winner_id, loser_id in data_reduced2018.itertuples():
    if (winner_id, loser_id) in G.edges:
        G.edges[winner_id, loser_id]['weight'] += 1
        G.edges[winner_id, loser_id]['god_2018'] += 1
    else:
        G.add_edge(winner_id, loser_id, weight = 1, god_2018 = 1, god_2019 = 0, god_2020 = 0)

for _, winner_id, loser_id in data_reduced2019.itertuples():
    if (winner_id, loser_id) in G.edges:
        G.edges[winner_id, loser_id]['weight'] += 1
        G.edges[winner_id, loser_id]['god_2019'] += 1
    else:
        G.add_edge(winner_id, loser_id, weight = 1, god_2018 = 0, god_2019 = 1, god_2020 = 0)
        
for _, winner_id, loser_id in data_reduced2020.itertuples():
    if (winner_id, loser_id) in G.edges:
        G.edges[winner_id, loser_id]['weight'] += 1
        G.edges[winner_id, loser_id]['god_2020'] += 1
    else:
        G.add_edge(winner_id, loser_id, weight = 1, god_2018 = 0, god_2019 = 0, god_2020 = 1)

In [7]:
output_path = "models/model_all_years.gml"
nx.write_gml(G, output_path)

In [8]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 581
Number of edges: 5330
Average degree:  18.3477


In [29]:
def plot_tournir_frequency(data, 
                           xscale = "linear", yscale = "linear"):

    if weighted:
        degrees = G.degree(weight="weight")
    else:
        degrees = G.degree()
    _, deg_list = zip(*degrees)
    deg_counts = Counter(deg_list)        
    print(deg_counts)
    x, y = zip(*deg_counts.items())                                                      

    plt.figure(1)   

    # prep axes   
    if weighted:
        plt.xlabel('weighted degree')  
    else:
        plt.xlabel('degree')                                                                                                             
    plt.xscale(xscale)                                                                                                                
    plt.xlim(1, max(x))  

    plt.ylabel('frequency')                                                                                                          
    plt.yscale(yscale)                                                                                                                
    plt.ylim(1, max(y))                                                                                                             
                                                                                                                                                                                                    
    plt.scatter(x, y, marker='.')                                                                                                    
    plt.show()

(7117, 49)
